In [ ]:
import sys
sys.path.append('../..')
from pymongo import UpdateOne
import matplotlib.pyplot as plt
from exploration.config import mongo_inst
from pymongo import MongoClient
from mlpp.data_collection.sample import osuDumpSampler

In [ ]:
client = MongoClient('localhost', 27017)

In [ ]:
db = client.osu_random_db

In [ ]:
data = mongo_inst["osu_random_db"]

In [ ]:
test = osuDumpSampler(data)

In [ ]:
test.use_random_sample("sample_scores_1M", 6500)

In [ ]:
data1 = mongo_inst["osu_random_db"]
test1 = osuDumpSampler(data1)

In [ ]:
collection = data["sample_scores_1M"]

In [ ]:
test.get_more_recent_than(collection, 2018, 12, 1, 0, 0, 0, "sample_scores_500k")

In [ ]:
# test1.use_random_sample("sample_scores_100", 100)

In [ ]:
collection2 = data["sample_scores_500k"]

In [ ]:
"""
max est user pp in collection of 500k
"""
cursor=db.sample_scores_500k.aggregate(
   [
     {
       "$group":
         {
           "_id": {},
           "max": { "$max": "$mlpp.est_user_pp" }
         }
     }
   ]
)
for document in cursor:
    print(document)
print(document['max'])
max_pp = document['max']

In [ ]:
a = 0
b = 100

while b <= max_pp:
    db.uniform_collection2.insert_many(
        collection2.aggregate([
        {
        '$match': {
            'mlpp.est_user_pp' : {
                '$gt': a,
                '$lt': b,
            }
        }
    },
    {'$sample': {
        'size': 800
    }
}
        
])
    )
    a = b
    b += 100

In [ ]:
# l = [x * 100 for x in range (115)]
# print(l)

In [ ]:
#once you have a uniform collection, use this to have a collection with the most 1000 popular maps:
db.uniform_collection2.aggregate([
    {
        '$group': {
            '_id': '$beatmap_id', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }, {
        '$limit': 1000
    }, {
        '$out': 'oneThousand_most_popular_maps2'
    }
])

## Objective 3: Realtionship as Score Decreases

In [ ]:
g=[]
h=[]
x=db.oneThousand_most_popular_maps2
x1=x.find({},{ "_id": 0, "count": 1}) #finds only the count column in the collection
y1=x.find({},{ "_id": 1, "count": 0})

# for i in x.find({},{ "_id": 0, "count": 1}):
#     print(i)

for i in x1:
    g.append(i['count'])
for i in x.find({},{ "_id": 1, "count": 0}):
    h.append(i['_id'])

plt.plot(g,'ro') #automatically index x 
plt.xlabel('index')
plt.ylabel('count')

In [ ]:
collections = db.list_collection_names() 
print ("collections:", collections, "\n")